In [1]:
import os
import cv2
import numpy as np
import pydicom
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import shutil
from ultralytics import YOLO
import yaml
import json
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

class Config:
    """Configuration parameters"""
    # Dataset paths
    DATASET_ROOT = "YOLOv8 Dataset"
    DICOM_TEST_DIR = "DICOM Data"
    
    # Working directory
    WORKING_DIR = "Output"
    
    # Model parameters - USING SEGMENTATION MODEL
    MODEL_NAME = "yolov8n-seg.pt"  # ✅ Changed to segmentation model
    EPOCHS = 100
    IMG_SIZE = 640
    BATCH_SIZE = 4
    
    # Output directories
    OUTPUT_DIR = WORKING_DIR
    MODEL_SAVE_PATH = f"{WORKING_DIR}/best_metacarpal_model.pt"
    RESULTS_DIR = f"{WORKING_DIR}/results"
    DATASET_COPY = f"{WORKING_DIR}/dataset_clean"
    
    # BHI calculation
    PIXEL_SPACING = 0.143  # mm per pixel

# Create directories
os.makedirs(Config.RESULTS_DIR, exist_ok=True)

# ============================================================================
# DATASET PREPARATION
# ============================================================================

def prepare_dataset_for_training():
    """Copy and prepare dataset in working directory"""
    print("\n" + "="*70)
    print("PREPARING DATASET FOR TRAINING")
    print("="*70)
    
    # Create clean dataset directory
    os.makedirs(Config.DATASET_COPY, exist_ok=True)
    
    total_copied = 0
    total_failed = 0
    
    # Process each split
    for split in ['train', 'valid', 'test']:
        src_images = os.path.join(Config.DATASET_ROOT, split, 'images')
        src_labels = os.path.join(Config.DATASET_ROOT, split, 'labels')
        
        if not os.path.exists(src_images):
            print(f"⚠️  {split} split not found, skipping...")
            continue
        
        dst_images = os.path.join(Config.DATASET_COPY, split, 'images')
        dst_labels = os.path.join(Config.DATASET_COPY, split, 'labels')
        
        os.makedirs(dst_images, exist_ok=True)
        os.makedirs(dst_labels, exist_ok=True)
        
        print(f"\nProcessing {split} split...")
        
        image_files = [f for f in os.listdir(src_images) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        copied = 0
        failed = 0
        
        for img_file in image_files:
            src_img_path = os.path.join(src_images, img_file)
            dst_img_path = os.path.join(dst_images, img_file)
            
            try:
                # Read with PIL (more robust)
                pil_img = Image.open(src_img_path)
                
                # Convert to RGB if needed
                if pil_img.mode != 'RGB':
                    pil_img = pil_img.convert('RGB')
                
                # Convert to numpy array
                img_array = np.array(pil_img)
                
                # Validate
                if img_array is None or img_array.size == 0:
                    raise ValueError("Empty image")
                
                if len(img_array.shape) != 3 or img_array.shape[2] != 3:
                    raise ValueError(f"Invalid shape: {img_array.shape}")
                
                # Save as JPG with high quality
                img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
                cv2.imwrite(dst_img_path, img_bgr, [cv2.IMWRITE_JPEG_QUALITY, 95])
                
                # Verify saved image
                test_read = cv2.imread(dst_img_path)
                if test_read is None:
                    raise ValueError("Failed to verify saved image")
                
                # Copy label
                label_file = os.path.splitext(img_file)[0] + '.txt'
                src_label = os.path.join(src_labels, label_file)
                dst_label = os.path.join(dst_labels, label_file)
                
                if os.path.exists(src_label):
                    shutil.copy2(src_label, dst_label)
                else:
                    print(f"  ⚠️  No label for {img_file}")
                
                copied += 1
                
            except Exception as e:
                print(f"  ❌ Failed: {img_file} - {e}")
                failed += 1
                
                # Remove if partially created
                if os.path.exists(dst_img_path):
                    os.remove(dst_img_path)
                continue
        
        print(f"  ✓ Copied: {copied}")
        if failed > 0:
            print(f"  ❌ Failed: {failed}")
        
        total_copied += copied
        total_failed += failed
    
    # Create data.yaml
    yaml_content = {
        'path': Config.DATASET_COPY,
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'nc': 3,
        'names': ['2nd Metacarpal bone', '3rd Metacarpal Bone', '4th Metacarpal bone']
    }
    
    yaml_path = os.path.join(Config.DATASET_COPY, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)
    
    print(f"\n{'='*70}")
    print(f"✓ Total images copied: {total_copied}")
    if total_failed > 0:
        print(f"⚠️  Total failed: {total_failed}")
    print(f"✓ Dataset saved to: {Config.DATASET_COPY}")
    print(f"{'='*70}")
    
    return yaml_path

# ============================================================================
# DICOM PROCESSING
# ============================================================================

class DICOMProcessor:
    """Handles DICOM file processing"""
    
    @staticmethod
    def read_dicom(dicom_path):
        """Read DICOM file"""
        try:
            dicom = pydicom.dcmread(dicom_path)
            image = dicom.pixel_array
            
            pixel_spacing = None
            if hasattr(dicom, 'PixelSpacing'):
                pixel_spacing = float(dicom.PixelSpacing[0])
            elif hasattr(dicom, 'ImagerPixelSpacing'):
                pixel_spacing = float(dicom.ImagerPixelSpacing[0])
            
            return image, pixel_spacing
        except Exception as e:
            print(f"Error reading DICOM: {e}")
            return None, None
    
    @staticmethod
    def normalize_image(image):
        """Normalize to 0-255"""
        if image is None:
            return None
        
        image = image.astype(np.float32)
        img_range = image.max() - image.min()
        if img_range == 0:
            return np.zeros_like(image, dtype=np.uint8)
        
        image = (image - image.min()) / img_range
        image = (image * 255).astype(np.uint8)
        return image
    
    @staticmethod
    def convert_to_rgb(image):
        """Convert to RGB"""
        if len(image.shape) == 2:
            return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        return image
    
    @staticmethod
    def preprocess_for_yolo(dicom_path, output_size=640):
        """Preprocess DICOM for YOLO"""
        image, pixel_spacing = DICOMProcessor.read_dicom(dicom_path)
        if image is None:
            return None, None
        
        image = DICOMProcessor.normalize_image(image)
        if image is None:
            return None, None
        
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        image = clahe.apply(image)
        
        image = DICOMProcessor.convert_to_rgb(image)
        
        h, w = image.shape[:2]
        scale = output_size / max(h, w)
        new_h, new_w = int(h * scale), int(w * scale)
        image = cv2.resize(image, (new_w, new_h))
        
        pad_h = (output_size - new_h) // 2
        pad_w = (output_size - new_w) // 2
        image = cv2.copyMakeBorder(image, pad_h, output_size-new_h-pad_h, 
                                   pad_w, output_size-new_w-pad_w, 
                                   cv2.BORDER_CONSTANT, value=(0,0,0))
        
        return image, pixel_spacing if pixel_spacing else Config.PIXEL_SPACING

# ============================================================================
# BHI CALCULATION - NEW FORMULA
# ============================================================================

class BHICalculator:
    """Calculate Bone Health Index using: BHI = πT(1−T/W)/(LW)^0.33"""
    
    @staticmethod
    def extract_bone_measurements(mask, pixel_spacing):
        """
        Extract comprehensive measurements from bone mask.
        Returns: (cortical_thickness, bone_width, bone_length)
        """
        if mask is None or mask.sum() == 0:
            return None, None, None
        
        mask = (mask > 0.5).astype(np.uint8)
        
        coords = np.where(mask > 0)
        if len(coords[0]) == 0:
            return None, None, None
        
        # Calculate bone length (vertical extent)
        min_y, max_y = coords[0].min(), coords[0].max()
        bone_length_pixels = max_y - min_y
        bone_length_mm = bone_length_pixels * pixel_spacing
        
        if bone_length_pixels < 10:
            return None, None, None
        
        # Define mid-diaphysis region (middle third)
        mid_start = min_y + bone_length_pixels // 3
        mid_end = max_y - bone_length_pixels // 3
        
        cortical_thicknesses = []
        bone_widths = []
        
        # Sample multiple points in mid-diaphysis
        measurement_points = max(10, min(30, (mid_end - mid_start) // 5))
        for y in np.linspace(mid_start, mid_end, measurement_points, dtype=int):
            if y >= mask.shape[0]:
                continue
                
            row = mask[y, :]
            if row.sum() == 0:
                continue
            
            bone_pixels = np.where(row > 0)[0]
            if len(bone_pixels) < 2:
                continue
            
            # Total bone width
            left_edge = bone_pixels[0]
            right_edge = bone_pixels[-1]
            width_mm = (right_edge - left_edge) * pixel_spacing
            
            if width_mm < 1:
                continue
                
            bone_widths.append(width_mm)
            
            # Cortical thickness estimation
            # Assume cortical bone is outer ~25% on each side
            bone_width_pixels = right_edge - left_edge
            cortical_region = max(1, int(bone_width_pixels * 0.25))
            
            # Average of left and right cortical thickness
            cortical_thickness = cortical_region * pixel_spacing
            cortical_thicknesses.append(cortical_thickness)
        
        if not cortical_thicknesses or not bone_widths:
            return None, None, None
        
        # Return averages
        avg_cortical = np.mean(cortical_thicknesses)
        avg_width = np.mean(bone_widths)
        
        return avg_cortical, avg_width, bone_length_mm
    
    @staticmethod
    def calculate_bhi(T, W, L):
        """
        Calculate BHI using new formula: BHI = πT(1−T/W)/(LW)^0.33
        
        Args:
            T: Average cortical thickness (mm)
            W: Average bone width (mm)
            L: Average bone length (mm)
        
        Returns:
            BHI value
        """
        if T is None or W is None or L is None:
            return None
        
        if W == 0 or L == 0:
            return None
        
        try:
            # BHI = πT(1−T/W)/(LW)^0.33
            numerator = np.pi * T * (1 - T/W)
            denominator = (L * W) ** 0.33
            
            if denominator == 0:
                return None
                
            bhi = numerator / denominator
            return bhi
            
        except Exception as e:
            print(f"Error calculating BHI: {e}")
            return None
    


# ============================================================================
# MODEL TRAINING
# ============================================================================

def train_model():
    """Train YOLOv8 model"""
    
    print("=" * 70)
    print("STARTING MODEL TRAINING - SEGMENTATION")
    print("="*70)
    print("⚠️  IMPORTANT: Using YOLOv8 SEGMENTATION model (yolov8n-seg.pt)")
    print("   This will use your polygon annotations to get accurate masks")
    print("="*70)
    
    # Prepare dataset
    yaml_path = prepare_dataset_for_training()
    
    # Verify dataset
    train_images = os.path.join(Config.DATASET_COPY, "train", "images")
    valid_images = os.path.join(Config.DATASET_COPY, "valid", "images")
    
    train_count = len([f for f in os.listdir(train_images) 
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    valid_count = len([f for f in os.listdir(valid_images) 
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    
    print(f"\n✓ Dataset ready")
    print(f"  Training images: {train_count}")
    print(f"  Validation images: {valid_count}")
    
    if train_count == 0:
        print("❌ No training images!")
        return None
    
    # Initialize segmentation model
    print(f"\n📦 Loading SEGMENTATION model: {Config.MODEL_NAME}")
    print(f"   This model will learn from your polygon annotations")
    model = YOLO(Config.MODEL_NAME)
    
    # Train on CPU
    print(f"\n🚀 Starting training on CPU...")
    print(f"  Epochs: {Config.EPOCHS}")
    print(f"  Image size: {Config.IMG_SIZE}")
    print(f"  Batch size: {Config.BATCH_SIZE}")
    print(f"  Device: CPU (No GPU detected)")
    print(f"  ⚠️  Training on CPU will be slower than GPU")
    
    try:
        results = model.train(
            data=yaml_path,
            epochs=Config.EPOCHS,
            imgsz=Config.IMG_SIZE,
            batch=Config.BATCH_SIZE,
            name='metacarpal_detection',
            patience=20,
            save=True,
            device='cpu',
            workers=2,
            plots=True,
            verbose=True,
            cache=False,
            rect=False,
            amp=False,
            lr0=0.01,
            lrf=0.01,
            momentum=0.937,
            weight_decay=0.0005,
            warmup_epochs=3.0,
            warmup_momentum=0.8,
            warmup_bias_lr=0.1,
            box=7.5,
            cls=0.5,
            dfl=1.5,
            hsv_h=0.015,
            hsv_s=0.7,
            hsv_v=0.4,
            degrees=0.0,
            translate=0.1,
            scale=0.5,
            shear=0.0,
            perspective=0.0,
            flipud=0.0,
            fliplr=0.5,
            mosaic=1.0,
            mixup=0.0,
            copy_paste=0.0
        )
        
        # Save best model
        best_model_path = model.trainer.best
        shutil.copy(best_model_path, Config.MODEL_SAVE_PATH)
        print(f"\n✓ Model saved: {Config.MODEL_SAVE_PATH}")
        
        # Validation
        metrics = model.val()
        
        print("\n" + "=" * 70)
        print("TRAINING COMPLETED")
        print("=" * 70)
        print(f"mAP50: {metrics.box.map50:.4f}")
        print(f"mAP50-95: {metrics.box.map:.4f}")
        print("=" * 70)
        
        return model
        
    except Exception as e:
        print(f"\n❌ Training error: {e}")
        import traceback
        traceback.print_exc()
        return None

# ============================================================================
# INFERENCE
# ============================================================================

class MetacarpalDetector:
    """Detection and BHI calculation pipeline"""
    
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.bone_names = ['2nd Metacarpal', '3rd Metacarpal', '4th Metacarpal']
    
    def detect_and_calculate_bhi(self, dicom_path, conf_threshold=0.25):
        """Complete pipeline with new BHI formula"""
        print(f"\n{'='*70}")
        print(f"Processing: {os.path.basename(dicom_path)}")
        print(f"{'='*70}")
        
        image, pixel_spacing = DICOMProcessor.preprocess_for_yolo(dicom_path)
        if image is None:
            print("❌ Failed to process DICOM")
            return None, None
        
        print(f"✓ Processed (spacing: {pixel_spacing:.4f} mm/px)")
        
        try:
            results = self.model(image, conf=conf_threshold, verbose=False)[0]
        except Exception as e:
            print(f"❌ Detection failed: {e}")
            return None, None
        
        boxes = results.boxes
        print(f"✓ Detected {len(boxes)} bones")
        
        if len(boxes) == 0:
            return results, {}
        
        # Collect measurements from all bones
        individual_measurements = {}
        all_T = []  # cortical thicknesses
        all_W = []  # bone widths
        all_L = []  # bone lengths
        
        for i, box in enumerate(boxes):
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            bone_name = self.bone_names[cls_id] if cls_id < len(self.bone_names) else f"Bone {cls_id}"
            
            print(f"\n--- {bone_name} (conf: {conf:.2%}) ---")
            
            # Extract mask - Segmentation model provides accurate masks
            if hasattr(results, 'masks') and results.masks is not None and len(results.masks.data) > i:
                # Get precise mask from segmentation
                mask = results.masks.data[i].cpu().numpy()
                mask = cv2.resize(mask, (image.shape[1], image.shape[0]))
                print(f"  ✓ Using segmentation mask (accurate)")
            else:
                # Fallback: This should NOT happen with segmentation model
                print(f"  ⚠️ WARNING: No mask available! Using bounding box fallback")
                print(f"     BHI measurements will be INACCURATE!")
                mask = np.zeros(image.shape[:2], dtype=np.uint8)
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)
                mask[y1:y2, x1:x2] = 1
            
            # Extract measurements
            T, W, L = BHICalculator.extract_bone_measurements(mask, pixel_spacing)
            
            if T is not None and W is not None and L is not None:
                individual_measurements[bone_name] = {
                    'cortical_thickness_mm': round(T, 3),
                    'bone_width_mm': round(W, 3),
                    'bone_length_mm': round(L, 3),
                    'confidence': round(conf, 4)
                }
                
                print(f"  T (cortical): {T:.3f}mm")
                print(f"  W (width): {W:.3f}mm") 
                print(f"  L (length): {L:.3f}mm")
                
                all_T.append(T)
                all_W.append(W)
                all_L.append(L)
            else:
                individual_measurements[bone_name] = {
                    'error': 'Measurement failed',
                    'confidence': round(conf, 4)
                }
                print(f"  ⚠️ Measurement failed")
        
        # Calculate BHI using averages across all detected bones
        bhi_result = {}
        
        if len(all_T) > 0 and len(all_W) > 0 and len(all_L) > 0:
            avg_T = np.mean(all_T)
            avg_W = np.mean(all_W)
            avg_L = np.mean(all_L)
            
            bhi = BHICalculator.calculate_bhi(avg_T, avg_W, avg_L)
            
            bhi_result = {
                'individual_bones': individual_measurements,
                'average_cortical_thickness_mm': round(avg_T, 3),
                'average_bone_width_mm': round(avg_W, 3),
                'average_bone_length_mm': round(avg_L, 3),
                'bhi': round(bhi, 4),
                'bones_measured': len(all_T),
                'formula': 'BHI = πT(1−T/W)/(LW)^0.33'
            }
            
            print(f"\n{'='*70}")
            print(f"BHI CALCULATION (using {len(all_T)} bones)")
            print(f"{'='*70}")
            print(f"Average T: {avg_T:.3f}mm")
            print(f"Average W: {avg_W:.3f}mm")
            print(f"Average L: {avg_L:.3f}mm")
            print(f"BHI: {bhi:.4f}")
            print(f"{'='*70}")
        else:
            bhi_result = {
                'individual_bones': individual_measurements,
                'error': 'Insufficient measurements for BHI calculation'
            }
            print(f"\n⚠️ Could not calculate BHI - insufficient valid measurements")
        
        return results, bhi_result
    
    def visualize_results(self, results, bhi_measurements, save_path=None):
        """Create visualization"""
        if results is None:
            return
        
        annotated_img = results.plot()
        
        # Add individual bone info
        y_offset = 30
        if 'individual_bones' in bhi_measurements:
            for bone_name, measurements in bhi_measurements['individual_bones'].items():
                if 'cortical_thickness_mm' in measurements:
                    text = f"{bone_name}: T={measurements['cortical_thickness_mm']:.2f}mm"
                    cv2.putText(annotated_img, text, (10, y_offset),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    y_offset += 22
        
        # Add BHI result
        if 'bhi' in bhi_measurements:
            y_offset += 10
            text = f"BHI: {bhi_measurements['bhi']:.4f}"
            cv2.putText(annotated_img, text, (10, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        plt.figure(figsize=(14, 10))
        plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title('Metacarpal Detection & BHI Analysis\nBHI = πT(1−T/W)/(LW)^0.33', 
                 fontsize=14, fontweight='bold')
        
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            print(f"✓ Saved: {save_path}")
        
        plt.show()
        plt.close()

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution"""
    
    print("\n" + "="*70)
    print("METACARPAL BONE HEALTH INDEX CALCULATOR")
    print("Using YOLOv8 SEGMENTATION for accurate measurements")
    print("Formula: BHI = πT(1−T/W)/(LW)^0.33")
    print("="*70)
    
    # Step 1: Training
    SKIP_TRAINING = False  # Set to True to skip training
    
    if not SKIP_TRAINING:
        print("\n[STEP 1] Training model...")
        model = train_model()
        if model is None:
            print("\n❌ Training failed")
            return
    else:
        print("\n[STEP 1] Skipping training")
    
    # Step 2: Load model
    print("\n[STEP 2] Loading model...")
    if not os.path.exists(Config.MODEL_SAVE_PATH):
        print(f"❌ Model not found: {Config.MODEL_SAVE_PATH}")
        return
    
    detector = MetacarpalDetector(Config.MODEL_SAVE_PATH)
    print(f"✓ Model loaded")
    
    # Step 3: Test on DICOMs
    print("\n[STEP 3] Testing on DICOM images...")
    
    if not os.path.exists(Config.DICOM_TEST_DIR):
        print(f"⚠️ DICOM directory not found: {Config.DICOM_TEST_DIR}")
        return
    
    dicom_files = [f for f in os.listdir(Config.DICOM_TEST_DIR) 
                   if f.lower().endswith('.dcm')]
    
    if not dicom_files:
        print("⚠️ No DICOM files found")
        return
    
    print(f"✓ Found {len(dicom_files)} files, processing first 10...")
    
    all_results = {}
    success_count = 0
    
    for idx, dicom_file in enumerate(dicom_files[:10], 1):
        print(f"\n[{idx}/10] {dicom_file}")
        dicom_path = os.path.join(Config.DICOM_TEST_DIR, dicom_file)
        
        results, bhi_measurements = detector.detect_and_calculate_bhi(dicom_path)
        
        if results is not None:
            save_path = os.path.join(Config.RESULTS_DIR, 
                                    f"{os.path.splitext(dicom_file)[0]}_result.png")
            detector.visualize_results(results, bhi_measurements, save_path)
            all_results[dicom_file] = bhi_measurements
            success_count += 1
        else:
            all_results[dicom_file] = {'error': 'Processing failed'}
    
    # Save JSON
    json_path = os.path.join(Config.RESULTS_DIR, "bhi_results.json")
    with open(json_path, 'w') as f:
        json.dump(all_results, f, indent=2)
    
    print(f"\n{'='*70}")
    print(f"✓ Processed {success_count}/10 images successfully")
    print(f"✓ Results: {json_path}")
    print(f"✓ Images: {Config.RESULTS_DIR}")
    print(f"{'='*70}")

# ============================================================================
# HELPER
# ============================================================================

def process_single_dicom(dicom_path, model_path=None):
    """Process single DICOM"""
    if model_path is None:
        model_path = Config.MODEL_SAVE_PATH
    
    if not os.path.exists(model_path):
        print(f"❌ Model not found: {model_path}")
        return None, None
    
    detector = MetacarpalDetector(model_path)
    results, bhi = detector.detect_and_calculate_bhi(dicom_path)
    
    if results is not None:
        detector.visualize_results(results, bhi)
    
    return results, bhi

# ============================================================================
# RUN
# ============================================================================

if __name__ == "__main__":
    main()


METACARPAL BONE HEALTH INDEX CALCULATOR
Using YOLOv8 SEGMENTATION for accurate measurements
Formula: BHI = πT(1−T/W)/(LW)^0.33

[STEP 1] Training model...
STARTING MODEL TRAINING - SEGMENTATION
⚠️  IMPORTANT: Using YOLOv8 SEGMENTATION model (yolov8n-seg.pt)
   This will use your polygon annotations to get accurate masks

PREPARING DATASET FOR TRAINING

Processing train split...
  ✓ Copied: 179

Processing valid split...
  ✓ Copied: 51

Processing test split...
  ✓ Copied: 26

✓ Total images copied: 256
✓ Dataset saved to: Output/dataset_clean

✓ Dataset ready
  Training images: 179
  Validation images: 51

📦 Loading SEGMENTATION model: yolov8n-seg.pt
   This model will learn from your polygon annotations

🚀 Starting training on CPU...
  Epochs: 100
  Image size: 640
  Batch size: 4
  Device: CPU (No GPU detected)
  ⚠️  Training on CPU will be slower than GPU
Ultralytics 8.3.231  Python-3.11.7 torch-2.9.1+cpu CPU (AMD Ryzen 5 5600G with Radeon Graphics)
engine\trainer: agnostic_nms=Fal

<Figure size 1400x1000 with 1 Axes>


[2/10] dicom_0002.dcm

Processing: dicom_0002.dcm
✓ Processed (spacing: 0.1400 mm/px)
✓ Detected 3 bones

--- 2nd Metacarpal (conf: 96.78%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.448mm
  W (width): 2.016mm
  L (length): 15.540mm

--- 4th Metacarpal (conf: 96.69%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.308mm
  W (width): 1.498mm
  L (length): 12.740mm

--- 3rd Metacarpal (conf: 94.33%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.420mm
  W (width): 1.722mm
  L (length): 14.420mm

BHI CALCULATION (using 3 bones)
Average T: 0.392mm
Average W: 1.745mm
Average L: 14.233mm
BHI: 0.3308
✓ Saved: Output/results\dicom_0002_result.png


<Figure size 1400x1000 with 1 Axes>


[3/10] dicom_0003.dcm

Processing: dicom_0003.dcm
✓ Processed (spacing: 0.1400 mm/px)
✓ Detected 3 bones

--- 2nd Metacarpal (conf: 96.27%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.687mm
  W (width): 2.851mm
  L (length): 22.820mm

--- 4th Metacarpal (conf: 95.86%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.476mm
  W (width): 2.142mm
  L (length): 19.460mm

--- 3rd Metacarpal (conf: 92.13%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.636mm
  W (width): 2.800mm
  L (length): 22.820mm

BHI CALCULATION (using 3 bones)
Average T: 0.600mm
Average W: 2.598mm
Average L: 21.700mm
BHI: 0.3831
✓ Saved: Output/results\dicom_0003_result.png


...


[9/10] dicom_0009.dcm

Processing: dicom_0009.dcm
✓ Processed (spacing: 0.1400 mm/px)
✓ Detected 3 bones

--- 2nd Metacarpal (conf: 96.66%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.875mm
  W (width): 3.698mm
  L (length): 25.060mm

--- 3rd Metacarpal (conf: 96.36%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.723mm
  W (width): 3.127mm
  L (length): 25.060mm

--- 4th Metacarpal (conf: 93.67%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.484mm
  W (width): 2.227mm
  L (length): 23.380mm

BHI CALCULATION (using 3 bones)
Average T: 0.694mm
Average W: 3.017mm
Average L: 24.500mm
BHI: 0.4058
✓ Saved: Output/results\dicom_0009_result.png


<Figure size 1400x1000 with 1 Axes>


[10/10] dicom_0010.dcm

Processing: dicom_0010.dcm
✓ Processed (spacing: 0.1400 mm/px)
✓ Detected 3 bones

--- 2nd Metacarpal (conf: 95.79%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.616mm
  W (width): 2.674mm
  L (length): 20.020mm

--- 4th Metacarpal (conf: 94.66%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.462mm
  W (width): 2.100mm
  L (length): 17.780mm

--- 3rd Metacarpal (conf: 93.78%) ---
  ✓ Using segmentation mask (accurate)
  T (cortical): 0.588mm
  W (width): 2.576mm
  L (length): 19.460mm

BHI CALCULATION (using 3 bones)
Average T: 0.555mm
Average W: 2.450mm
Average L: 19.087mm
BHI: 0.3793
✓ Saved: Output/results\dicom_0010_result.png


<Figure size 1400x1000 with 1 Axes>


✓ Processed 10/10 images successfully
✓ Results: Output/results\bhi_results.json
✓ Images: Output/results
